In [1]:
import requests
from bs4 import BeautifulSoup
import re
import json
from selenium import webdriver
from lxml import etree
import os
from os import path
import shutil
import time
import demjson
from selenium.webdriver.chrome.options import Options
from multiprocessing.dummy import Pool

In [2]:
def getResponse(url):
    headers = {
        'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.182 Safari/537.36'
    }
    response = requests.get(url=url, headers=headers)
    return response

In [3]:
def getHTML(url):
    response = getResponse(url)
    response.encoding = 'utf-8'
    return response.text

In [4]:
# 通过selenium获取page_text
def getHTMLBySelenium(url):
    chrome_options =Options()
    chrome_options.add_argument('--headless')
    browser = webdriver.Chrome(options=chrome_options)
    browser.get(url)
    time.sleep(2)
    page_text = browser.page_source
    browser.quit()
    return page_text

In [4]:
# https://v.163.com/static/3/VB8TUVVKN.html
# http://flv.bn.netease.com/d954d53105dc303ad5e7acd5feea841357a0a32d97d998d71d7ff6889ccbe7093823148f822176b5c99dd5c6a0fd50a57ed5af868fe2677a935459a624a707c8e3ef3c2ba8042a268b2720a897c66cf75d89341fce1ca3c9d631001b1fc41a2d631af8fe70e31d348f7a5d48c140d421a01fd3bf6c9e073a.mp4

In [5]:
def getSrcUrl(url):
    page_text = getHTMLBySelenium(url)
    tree = etree.HTML(page_text)
    srcUrl = tree.xpath('//video/@src')
    if len(srcUrl) == 0:
        return ""
    #print(srcUrl)
    return srcUrl[0]

In [12]:
print(getSrcUrl('https://v.163.com/static/3/VB8VRCQSG.html'))

http://flv.bn.netease.com/d954d53105dc303ad422b5cb02c6d79055b43c9190c13f4b70c953d45d3bf23a7c4b7669b02f422a5bf562072310f149cb4be450adaa1761fbfcdb565061c13d47c8806aef764f8eda44bafc862930ccfc32588bc3341d32af2a7f9ef02e8942184ab5f75395cc7e33bcc5804cad6a21abc4819d5f8a19ac.mp4


In [6]:
# http://money.163.com/special/yuanchuang/
url = 'http://money.163.com/special/yuanchuang/'
page_text = getHTMLBySelenium(url)
tree = etree.HTML(page_text)
all_ul = tree.xpath('//div[@class="videolist"]/ul')
print(len(all_ul))

6


In [7]:
pageUrls = []
for ul in all_ul:
    all_li = ul.xpath('./li')
    print(len(all_li))
    for li in all_li:
        pageUrl = li.xpath('./a/@href')
        if len(pageUrl) == 0:
            continue
        print(pageUrl[0])
        pageUrls.append(pageUrl[0])

6
http://v.money.163.com/video/2015/12/K/N/VB8TUVVKN.html
http://v.money.163.com/video/2015/10/U/P/VB5TRUJUP.html
http://v.money.163.com/video/2015/12/S/G/VB8VRCQSG.html
http://v.money.163.com/video/2014/9/8/2/VA49VH082.html
http://v.money.163.com/video/2014/10/H/S/VA7OFHPHS.html
http://v.money.163.com/video/2014/8/G/H/VA3O0R5GH.html
6
http://v.money.163.com/video/2014/9/C/5/VA4CV5GC5.html
http://v.money.163.com/video/2014/11/F/0/VAAJ2FLF0.html
http://v.money.163.com/video/2014/11/L/J/VAAGJLQLJ.html
http://v.money.163.com/video/2014/8/I/V/VA3QUHSIV.html
http://v.money.163.com/video/2014/8/7/5/VA2H6BD75.html
http://v.money.163.com/video/2014/8/M/P/VA1VCCDMP.html
6
http://v.money.163.com/video/2014/6/J/Q/V9UODFLJQ.html
http://v.money.163.com/video/2014/5/G/N/V9SDMU7GN.html
http://v.money.163.com/video/2014/5/J/2/V9ROUNAJ2.html
http://v.money.163.com/video/2014/5/U/7/V9R6H66U7.html
http://v.money.163.com/video/2014/5/9/9/V9QL2S399.html
http://v.money.163.com/video/2014/3/7/P/V9NG6257P.htm

In [8]:
print(len(pageUrls))

36


In [9]:
with open('pageUrl.txt', 'w', encoding='utf-8') as fp:
    data = '\n'.join(pageUrls)
    fp.write(data)

In [10]:
srcUrls = []
for pageUrl in pageUrls:
    srcUrl = getSrcUrl(pageUrl)
    print(srcUrl)
    srcUrls.append(srcUrl)

http://flv.bn.netease.com/d954d53105dc303ae2a405a0c5c98113877a129e2a24875b3d9c92f4dcdee7268ca6a82a76de81a4755f37bd210dfbea0a09b5f6bed7e34c7e411736002dfb4ce5db0bba5864db768d8dbeb142dbbee9ec023697dd36f2ef9c0075784079846faae1968f0014f4762b38e5dba0f6c4a9b431f279c6b54f07.mp4
http://flv.bn.netease.com/d954d53105dc303a08a8537be81814da4d9ccdfaf6876539db5c82f88415f4f228117f586ce28a52b61bac3290946494da21ad06ac8f9b847bcfa6abf022b5e554e7efc8a8d90c1fb51935eb45455104010a34f42bd12fc62e21bf3837d0a3e24810a959100ec6fdd579ec93eff7fa4cd9f24efc065961a8.mp4
http://flv.bn.netease.com/d954d53105dc303a5ea171f4129a5926722057cf95625dcee90955d03465b8120a747cf5fb773dabf6f44268438224585544718ae6d08add046a16a4d8866a8b4bfb0b0498ba4df8c7df4ea6edae9052e7e7880729c5f68338e6d6bcdb6b738e153b97f407eac42a506c1ba5ba2feafc13ac0b2c33ebc790.mp4
http://flv.bn.netease.com/d954d53105dc303a5d630d586c9509fbe998d13ed4f9bdae3fe0b54b78e729de1182283b00069681645cde63e9c98705a47e3f4a46440118018adcea9c20d5404f16668aa6e99385001640a0ebe10944a

http://flv.bn.netease.com/d954d53105dc303af6207cfd93ee2e8ac57f479406e9308d994198f5c30fcbacac09a2837390a27d58879e9513e807c2a7bb96931f816118ad5cea5e60c0be01f540fd1ade3205653bcf43659776bc6bf842fa9426e9a3b21204e5987f388a60fd7d79772346903c1df6f6410cf31d6a374e07c496d8cf15.mp4
http://flv.bn.netease.com/d954d53105dc303ac8907a8a919a066ffb437812be710a9cd8a7246205550411bb09a03db5a8b055e2cdb556be16a6e796681539d96c298550683a3506f70d0ba7591b61977b136a232de6b82d716ea57403d4e1a63361b758d44cd0bdc380895463d8fd4ddbebf2f8c7db64fe6807b655649e7b0ed8d22b.mp4
http://flv.bn.netease.com/d954d53105dc303ada4cdc922ab6affc52e04d9c4a4589fb4d24043b4734a5e3b546c8f4208e66a93d95905cb16c5ef827053fb0f76fb36a40ba104ac054237e0f9b744bec07b78a1d23edfb90f0636f5da747fbd770b94c6cb54990ecc24bf3bf3a478c9cb70f5e0d6991c9311909882e45ffce68fc1145.mp4
http://flv.bn.netease.com/d954d53105dc303a8a78208a07b494150a32c71567c28b9e24dcc63690bcddff08c163f103665366755bd0976b7d7f1b0ed2a10396a56e3c952c3eb5122110a6debf3ac98ac3828f13f48c7fb02dff9e0

In [11]:
with open('srcUrl.txt', 'w', encoding='utf-8') as fp:
    data = '\n'.join(srcUrls)
    fp.write(data)

In [21]:
def download(url, fileName):
    #fileName = url.rsplit('/', 1)[-1]
    #print(fileName)
    path = 'video/'
    try:        
        content = getResponse(url).content
        with open(path + fileName, 'wb') as fp:
            fp.write(content)
        print(fileName, 'successful')
    except Exception as e:
        print("error")

In [13]:
download('http://flv.bn.netease.com/d954d53105dc303aee66920b48b99133bc2e9b3fcde4344effca8ac90dcea01d243390271664159273304838e441fb28ecdd740d2e16c74bf389f4655ed150745c6937fa8b152400808e320d36aee13fa4a6f048fe8a56c2635b5c93c6c67d54b4488fd32cbd93a5afe5cd8a4c84210883b8fc5fa10c052d.mp4')

d954d53105dc303aee66920b48b99133bc2e9b3fcde4344effca8ac90dcea01d243390271664159273304838e441fb28ecdd740d2e16c74bf389f4655ed150745c6937fa8b152400808e320d36aee13fa4a6f048fe8a56c2635b5c93c6c67d54b4488fd32cbd93a5afe5cd8a4c84210883b8fc5fa10c052d.mp4 successful


In [14]:
allObj = []
for i, url in enumerate(srcUrls):
    obj = {
        'id':i,
        'url':url
    }
    allObj.append(obj)
print(len(allObj))

36


In [20]:
def multithreadDownload(obj):
    ID = obj['id']
    url = obj['url']
    print(f'start download {ID}')
    download(url, str(ID)+'.mp4')

In [22]:
pool = Pool(4)
pool.map(multithreadDownload, allObj)
pool.close()
pool.join()

start download 0
start download 3
start download 6
start download 9
0.mp4 successful
start download 1
9.mp4 successful
start download 10
10.mp4 successful
start download 11
3.mp4 successful
start download 4
6.mp4 successful
start download 7
4.mp4 successful
start download 5
1.mp4 successful
start download 2
5.mp4 successful
start download 12
7.mp4 successful
start download 8
12.mp4 successful
start download 13
8.mp4 successful
start download 15
2.mp4 successful
start download 18
11.mp4 successful
start download 21
18.mp4 successful
start download 19
15.mp4 successful
start download 16
16.mp4 successful
start download 17
21.mp4 successful
start download 22
13.mp4 successful
start download 14
17.mp4 successful
start download 24
24.mp4 successful
start download 25
14.mp4 successful
start download 27
19.mp4 successful
start download 20
22.mp4 successful
start download 23
27.mp4 successful
start download 28
28.mp4 successful
start download 29
23.mp4 successful
start download 30
25.mp4 succe